# Selenium
**Selenium** es una _Web Browser Automation Tool_, es decir, permite abrir un buscador y realizar tareas que "imiten" el comportamiento humano (clicks, scrolling, *búsquedas*, etc).  
Link a la documentación de Selinium: [https://selenium-python.readthedocs.io/](https://selenium-python.readthedocs.io/)  
Link a la dicumentación y descarga de chromedriver: [https://chromedriver.chromium.org/home](https://chromedriver.chromium.org/home)

In [1]:
# !pip install selenium

In [2]:
# !apt-get install -y chromium-browser

In [3]:
""" !apt-get update
# !apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') """

" !apt-get update\n# !apt install chromium-chromedriver\n!cp /usr/lib/chromium-browser/chromedriver /usr/bin\nimport sys\nsys.path.insert(0,'/usr/lib/chromium-browser/chromedriver') "

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ActionChains

from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC

from tqdm import tqdm

import pandas as pd
import time

In [5]:
from selenium import webdriver

def configure_driver():
    # Add additional Options to the webdriver
    options = webdriver.ChromeOptions()
    # add the argument and make the browser Headless.
    # options.add_argument('--headless')
    # options.add_argument('--no-sandbox') # only colab
    # options.add_argument('--disable-dev-shm-usage') # only colab
    # Handle the service
    # service = Service(executable_path=r'/usr/bin/chromedriver')
    # Instantiate the Webdriver: Mention the executable path of the webdriver you have downloaded
    driver = webdriver.Chrome(options=options)

    #driver = webdriver.Firefox()
    return driver

In [6]:
def getCourses(driver, total_clicks):
    # https://www.pluralsight.com/search?q=web%20scraping&categories=course
    # Step 1: Go to pluralsight.com, category section with selected search keyword
    # driver.get("https://www.pluralsight.com/search?q={search_keyword}&categories=course")
    driver.get("https://www.pluralsight.com/search?categories=course&sort=displayDate")

    timeout = 20
    # time.sleep(timeout)
    count = 0
    loop_obj = tqdm(range(total_clicks))
    for k in loop_obj:
        # loop_obj.set_description(f"Epoch: {k + 1}")
        try:
            element = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((By.XPATH, "//a[@id='search-results-section-load-more']")))
            element.click()
            count = 0
        except TimeoutException as r:
            count += 1
            # print("Error:", count)
            if count > 5:
                break
        loop_obj.set_postfix_str(f"Count error: {count}")

    data = []
    # wait for the element to load
    try:
        WebDriverWait(driver, timeout*10).until(lambda s: s.find_element(By.ID, "search-results-category-target").is_displayed())
    except TimeoutException:
        print("Timeout :(")
        return None
    
    # Step 2: Create a parse tree of page sources after searching
    soup = BeautifulSoup(driver.page_source, "lxml")

    # Step 3: Iterate over the search result and fetch the course
    print(len(soup.select("div.search-results-page")))
    for course_page in tqdm(soup.select("div.search-results-page")):
        for course in course_page.select("div.search-result"):
            title_selector = "div.search-result__info div.search-result__title a"
            title_url_selector = "div.search-result__info div.search-result__title a"
            title_badge_selector = "div.search-result__info div.search-result__title span"
            author_selector = "div.search-result__details div.search-result__author"
            level_selector = "div.search-result__details div.search-result__level"
            length_selector = "div.search-result__details div.search-result__length"
            date_selector = "div.search-result__details div.search-result__date"
            icon_selector = "div.search-result__icon img"

            data.append({
                "title" : course.select_one(title_selector).text,
                "title_url" : course.select_one(title_url_selector).attrs["href"],
                "title_badge" : course.select_one(title_badge_selector).text,
                "date": course.select_one(date_selector).text,
                "author" : course.select_one(author_selector).text,
                "level" : course.select_one(level_selector).text,
                "length" : course.select_one(length_selector).text,
                "icon": course.select_one(icon_selector).attrs["src"]
            })
    return data

total_clicks = 500
# create the driver object.
driver = configure_driver()
data = getCourses(driver, total_clicks)


 81%|████████  | 405/500 [09:57<02:20,  1.47s/it, Count error: 5]


400


100%|██████████| 400/400 [00:05<00:00, 79.04it/s]


In [14]:
# close the driver.
driver.close()

In [7]:
df_data = pd.DataFrame.from_dict(data)
df_data

,title,title_url,title_badge,date,author,level,length,icon
0,Introduction to Microsoft Copilot,https://www.pluralsight.com/courses/microsoft-...,Skills - Expanded,Sep 1 2023,by Pluralsight,Beginner,1h 26m,https://pluralsight.imgix.net/course-images/mi...
1,Introduction to the Microsoft Collaboration Co...,https://www.pluralsight.com/courses/msft-colla...,Skills - Expanded,Sep 1 2023,by Pluralsight,Beginner,29m,https://pluralsight.imgix.net/course-images/ms...
2,Key Concepts: Data Architecture Design,https://www.pluralsight.com/courses/data-archi...,Skills - Expanded,Sep 1 2023,by Pluralsight,Beginner,1h 27m,https://pluralsight.imgix.net/course-images/da...
3,Introduction to Emotional Intelligence,https://www.pluralsight.com/courses/introducti...,Skills - Expanded,Sep 1 2023,by Pluralsight,Beginner,1h 27m,https://pluralsight.imgix.net/course-images/in...
4,Functional Programming in Rust 2021,https://www.pluralsight.com/courses/rust-2021-...,Skills - Expanded,Sep 1 2023,by Pluralsight,Intermediate,1h 14m,https://pluralsight.imgix.net/course-images/ru...
...,...,...,...,...,...,...,...,...
9995,Geospatial Mapping with D3,https://www.pluralsight.com/courses/geospatial...,Skills - Expanded,May 9 2016,by Ben Sullins,Intermediate,2h 38m,https://pluralsight.imgix.net/course-images/ge...
9996,Connecting Our Macs to Directory Services,https://www.pluralsight.com/courses/connecting...,Skills - Expanded,May 9 2016,by Ken Mauldin,Beginner,2h 25m,https://pluralsight.imgix.net/course-images/co...
9997,Visual Programming Introduction with Dynamo an...,https://www.pluralsight.com/courses/dynamo-rev...,Skills - Expanded,May 6 2016,by Kevin Griendling,Intermediate,2h 9m,https://pluralsight.imgix.net/course-images/dy...
9998,Creating a Photorealistic Scene with 3ds Max a...,https://www.pluralsight.com/courses/3ds-max-v-...,Skills - Expanded,May 5 2016,by Zaheer Mukhtar,Intermediate,4h 54m,https://pluralsight.imgix.net/course-images/3d...


In [8]:
df_data.to_csv("10000_dataset_cursos_{}.csv".format(total_clicks))

# **Handling AJAX Loading and Infinite Loading**
Sometimes, fetching content from dynamic sites is actually straightforward, as they are highly dependent on API calls. In asynchronous loading, most of the time, data is loaded by making GET and POST requests; you can watch these API calls in the Network tab of Developer Tools.
https://www.premierleague.com/players


In [ ]:
import requests
import json

In [ ]:
# https://footballapi.pulselive.com/football/players?pageSize=30&compSeasons=489&altIds=true&page=0&type=player&id=-1&compSeasonId=489

In [ ]:
# API url
url = "https://footballapi.pulselive.com/football/players"
# Headers required for making a GET request
# It is a good practice to provide headers with each request.
headers = {
    "account": "premierleague",
    "content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "DNT": "1",
    "Origin": "https://www.premierleague.com",
    "Referer": "https://www.premierleague.com/players",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"
}

# Query parameters required to make get request
queryParams = {
    "pageSize": 100,
    "compSeasons": 578,
    "altIds": True,
    "page": 0,
    "type": "player",
    "compSeasonId": 578,
    "cl":2,
}

In [ ]:
# Sending the request with url, headers, and query params
response = requests.get(url=url, headers=headers, params=queryParams)

In [ ]:
response.status_code

200

In [ ]:
response.text

<bound method Response.json of <Response [200]>>

In [ ]:
# if response status code is 200 OK, then
if response.status_code == 200:
  data = json.loads(response.text)
  for player in data["content"]:
    print({
        "name": player["name"]["display"],
        "nationalTeam": player["nationalTeam"]["country"],
        "position": player["info"]["positionInfo"]
    })

{'name': 'Max Aarons', 'nationalTeam': 'England', 'position': 'Right Full Back'}
{'name': 'Zach Abbott', 'nationalTeam': 'England', 'position': 'Defender'}
{'name': 'Terry Ablade', 'nationalTeam': 'Finland', 'position': 'Centre Striker'}
{'name': 'Tyler Adams', 'nationalTeam': 'United States', 'position': 'Midfielder'}
{'name': 'Tosin Adarabioyo', 'nationalTeam': 'England', 'position': 'Centre Central Defender'}
{'name': 'Tayo Adaramola', 'nationalTeam': 'Ireland', 'position': 'Left Full Back'}
{'name': 'Elijah Adebayo', 'nationalTeam': 'England', 'position': 'Centre Striker'}
{'name': 'Valintino Adedokun', 'nationalTeam': 'Ireland', 'position': 'Defender'}
{'name': 'Simon Adingra', 'nationalTeam': 'Cote D’Ivoire', 'position': 'Left Winger'}
{'name': 'Adler Nascimento', 'nationalTeam': 'Portugal', 'position': 'Left Winger'}
{'name': 'Adrián', 'nationalTeam': 'Spain', 'position': 'Goalkeeper'}
{'name': 'Daniel Adu-Adjei', 'nationalTeam': 'England', 'position': 'Midfielder'}
{'name': 'Na